# Data Cleaning and Visualization - Lite Version

This notebook contains all the Python code from the data cleaning slides.

## Step 0: Load Python's Libraries

In [ ]:
# ========================================
# Import Required Libraries
# ========================================
# pandas (pd): The primary library for working with tabular data (like spreadsheets)
import pandas as pd

# numpy (np): Library for numerical computations and array operations
import numpy as np

# matplotlib.pyplot (plt): The main plotting library for creating visualizations
import matplotlib.pyplot as plt

## Step 1 (a): Load Your Data From a File

In [ ]:
# Load your dataset
df = pd.read_csv('your_data.csv')

# First look at the data
df.head()

## Step 1 (b): Prepare (Random) Data

In [ ]:
# datetime modules: For working with dates and times
from datetime import datetime, timedelta

# ========================================
# Configure Settings
# ========================================
# Set random seed to 42 so everyone gets the same "random" data
# This makes our code reproducible - you'll get the same results every time
np.random.seed(42)

# Configure pandas display options for better readability
pd.set_option('display.max_columns', None)  # Show all columns (don't truncate)
pd.set_option('display.width', None)        # Use full screen width

print("✓ Libraries imported successfully!")
print("✓ Settings configured!")

## Step 1: Create The Dataset

In [ ]:
# ========================================
# Create Sample Dataset with Intentional Data Quality Issues
# ========================================
# We're creating a realistic dataset that has common problems you'll encounter in real data

# Set the number of records we want to generate
n_records = 500

# ----------------------------------------
# Generate Random Dates
# ----------------------------------------
# Create a starting date (January 1, 2024)
start_date = datetime(2024, 1, 1)

# Generate 500 random dates throughout 2024
# This simulates transaction dates spread across the year
dates = [start_date + timedelta(days=np.random.randint(0, 365)) for _ in range(n_records)]

# ----------------------------------------
# Build the Dataset Dictionary
# ----------------------------------------
# Create a dictionary where each key is a column name and value is a list of data
data = {
    # Transaction date for each sale
    'date': dates,
    
    # Product names - randomly chosen from 6 different products
    'product': np.random.choice(['Laptop', 'Phone', 'Tablet', 'Monitor', 'Keyboard', 'Mouse'], n_records),
    
    # Region names - NOTE: Intentionally inconsistent capitalization ('north' vs 'North' vs 'SOUTH')
    # This is a common data quality issue we'll need to fix!
    'region': np.random.choice(['North', 'South', 'East', 'West', 'north', 'SOUTH'], n_records),
    
    # Sales amount in dollars - random values between $100 and $5,000
    'sales': np.random.randint(100, 5000, n_records),
    
    # Quantity of items sold - random values between 1 and 50
    'quantity': np.random.randint(1, 50, n_records),
    
    # Customer age - random values between 18 and 75
    'customer_age': np.random.randint(18, 75, n_records),
    
    # Customer satisfaction score (1-5 scale, where 5 is best)
    'satisfaction': np.random.choice([1, 2, 3, 4, 5], n_records)
}

# Convert the dictionary into a pandas DataFrame (like a spreadsheet table)
df = pd.DataFrame(data)

# ----------------------------------------
# Introduce Missing Values (10% of data)
# ----------------------------------------
# Randomly select 10% of rows to have missing data
missing_indices = np.random.choice(df.index, size=int(n_records * 0.10), replace=False)

# Make half of those rows have missing sales values
df.loc[missing_indices[:len(missing_indices)//2], 'sales'] = np.nan

# Make the other half have missing satisfaction scores
df.loc[missing_indices[len(missing_indices)//2:], 'satisfaction'] = np.nan

# ----------------------------------------
# Introduce Duplicate Rows
# ----------------------------------------
# Randomly select 20 rows and duplicate them
duplicate_rows = df.sample(20)
# Add these duplicate rows to the dataframe (this creates duplicates)
df = pd.concat([df, duplicate_rows], ignore_index=True)

# ----------------------------------------
# Introduce Outliers
# ----------------------------------------
# Select 10 random rows and give them unrealistically high sales values
outlier_indices = np.random.choice(df.index, size=10, replace=False)
# Set their sales to be between $50,000 and $100,000 (much higher than normal)
df.loc[outlier_indices, 'sales'] = np.random.randint(50000, 100000, len(outlier_indices))

# ----------------------------------------
# Display Results
# ----------------------------------------
print(f"✓ Dataset created with {len(df)} records")
print(f"✓ Includes: missing values, duplicates, outliers, and inconsistent data")
print(f"\nFirst 10 rows of the dataset:")
df.head(10)

## Step 2: Check Data Structure

In [ ]:
# Get basic information
df.info()

# Check dimensions
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

# View column names and types
df.dtypes

## Step 3: Checking for Missing Values

In [ ]:
# Count missing values
missing = df.isnull().sum()

# Calculate percentage missing
missing_pct = (df.isnull().sum() / len(df)) * 100

# Display summary
pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})

## Step 3: Checking for Duplicates

In [ ]:
# Count duplicate rows
print(f"Duplicates: {df.duplicated().sum()}")

# View duplicate rows
df[df.duplicated(keep=False)]

## Step 3: Checking for Inconsistencies

In [ ]:
# Look at unique values in categorical columns
df['region'].value_counts()

# Part 2: Data Cleaning

## Step 1: Remove Duplicates

In [ ]:
# Create a copy (preserve original!)
df_clean = df.copy()

# Remove duplicates
df_clean = df_clean.drop_duplicates()

print(f"Removed {len(df) - len(df_clean)} duplicates")

## Step 2: Standardize Categorical Data

In [ ]:
# Fix inconsistent capitalization
df_clean['region'] = df_clean['region'].str.title()

# Before: 'north', 'NORTH', 'North'
# After:  'North', 'North', 'North'

## Step 3: Filling Missing Values - Example 1

In [ ]:
# Fill missing sales with median by product
df_clean['sales'] = df_clean.groupby('product')['sales'].transform(
    lambda x: x.fillna(x.median())
)

## Step 3: Filling Missing Values - Example 2

In [ ]:
# Fill satisfaction scores with mode
mode_value = df_clean['satisfaction'].mode()[0]
df_clean['satisfaction'] = df_clean['satisfaction'].fillna(mode_value)

## Step 4: Identify Outliers (IQR Method)

In [ ]:
Q1 = df_clean['sales'].quantile(0.25)
Q3 = df_clean['sales'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Find outliers
outliers = df_clean[
    (df_clean['sales'] < lower_bound) | 
    (df_clean['sales'] > upper_bound)
]

## Step 5: Handle Outliers - Option 1 (Remove Them)

In [ ]:
df_clean = df_clean[
    (df_clean['sales'] >= lower_bound) & 
    (df_clean['sales'] <= upper_bound)
]

## Step 5: Handle Outliers - Option 2 (Cap Them)

In [ ]:
df_clean['sales'] = df_clean['sales'].clip(
    lower=lower_bound, 
    upper=upper_bound
)

## Step 6: Add Derived Columns

In [ ]:
# Calculate price per unit
df_clean['price_per_unit'] = df_clean['sales'] / df_clean['quantity']

# Extract time features
df_clean['month'] = pd.to_datetime(df_clean['date']).dt.month
df_clean['quarter'] = pd.to_datetime(df_clean['date']).dt.quarter

# Create age groups
df_clean['age_group'] = pd.cut(
    df_clean['customer_age'], 
    bins=[0, 25, 35, 50, 100],
    labels=['18-25', '26-35', '36-50', '50+']
)

# Part 3: Data Subsetting

## Filtering by Single Condition

In [ ]:
# Filter by product
laptops = df_clean[df_clean['product'] == 'Laptop']

# Filter by value range
high_sales = df_clean[df_clean['sales'] > 2000]

## Filtering by Multiple Conditions

In [ ]:
# AND condition: both must be true
premium = df_clean[
    (df_clean['satisfaction'] >= 4) & 
    (df_clean['sales'] > 2000)
]

# OR condition: either can be true
tech_products = df_clean[
    (df_clean['product'] == 'Laptop') | 
    (df_clean['product'] == 'Phone')
]

## Using .isin() for Multiple Values

In [ ]:
# Filter for multiple categories
high_value = df_clean[
    df_clean['product'].isin(['Laptop', 'Monitor'])
]

# Filter by multiple regions and quarters
subset = df_clean[
    (df_clean['quarter'].isin([1, 2])) & 
    (df_clean['region'].isin(['North', 'East']))
]

# Best Practices

## Validate Your Cleaning

In [ ]:
# Check for remaining issues
assert df_clean.duplicated().sum() == 0
assert df_clean.isnull().sum().sum() == 0

# Verify data makes sense
print(f"Sales range: ${df_clean['sales'].min()} - ${df_clean['sales'].max()}")
print(f"Unique regions: {df_clean['region'].nunique()}")

## Document Your Decisions

In [ ]:
# Document your cleaning steps
cleaning_log = {
    'duplicates_removed': 20,
    'missing_sales_filled': 'median by product',
    'missing_satisfaction_filled': 'mode',
    'outliers_removed': 10,
    'reason': 'Values exceeded 3x IQR'
}

# Visualization

## Visualizing Data Quality

In [ ]:
import matplotlib.pyplot as plt

# Missing values heatmap
plt.figure(figsize=(10, 6))
plt.imshow(df.isnull(), cmap='viridis', aspect='auto')
plt.title('Missing Values Pattern')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.show()

## Before/After Comparison

In [ ]:
# Compare distributions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Before
df['sales'].hist(ax=ax1, bins=30)
ax1.set_title('Sales Distribution - Before Cleaning')

# After
df_clean['sales'].hist(ax=ax2, bins=30)
ax2.set_title('Sales Distribution - After Cleaning')

plt.show()

---

## Summary

You've completed the data cleaning workflow! Your data is now:
- Free of duplicates
- Standardized
- Free of missing values
- Outliers handled
- Enhanced with derived features

**Remember:** Clean data = Quality analysis = Better insights